# Initialisation et Chargement des Iris


In [3]:
from IPython.display import Image
%matplotlib inline

In [4]:
# Added version check for recent scikit-learn 0.18 checks
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

Loading the Iris dataset from scikit-learn.

In [5]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data
y = iris.target

print iris.data.shape

print('Class labels:', np.unique(y))

(150, 4)
('Class labels:', array([0, 1, 2]))


Splitting data into 70% training and 30% test data:

In [6]:
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

Standardizing the features:

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Le Perceptron

Redefining the `plot_decision_region` function:

In [8]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(n_iter=40, eta0=0.1, random_state=0)
ppn.fit(X_train_std, y_train)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=40, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [10]:
print y_test.shape

(45,)


In [11]:
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

Misclassified samples: 8


In [12]:
from sklearn.metrics import accuracy_score

print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))


print ppn.score(X_test_std,y_test )

Accuracy: 0.82
0.822222222222


Training a perceptron model using the standardized training data:

In [13]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

# K plus proches voisins

In [11]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')
knn.fit(X_train_std, y_train)

print knn.score(X_test_std, y_test)

0.933333333333


# Arbres de décision

In [12]:
from sklearn.tree import DecisionTreeClassifier

#tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)

tree = DecisionTreeClassifier(criterion='gini', splitter='best', )

tree.fit(X_train_std, y_train)

print('Accuracy: %.2f' % tree.score(X_test_std,y_test ))

print X_train_std.shape

X_combined = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

Accuracy: 0.98
(105, 4)


#  Régression logistique

In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=0)
lr.fit(X_train_std, y_train)


print('Accuracy: %.2f' % lr.score(X_test_std,y_test ))

Accuracy: 0.98


In [14]:
if Version(sklearn_version) < '0.17':
    lr.predict_proba(X_test_std[0, :])
else:
    lr.predict_proba(X_test_std[0, :].reshape(1, -1))

# Support vector machines

## Le cas non linéairement séparable et les slack variables

In [15]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(X_train_std, y_train)


print('Accuracy: %.2f' % svm.score(X_test_std,y_test ))

Accuracy: 0.98


# Travail à réaliser

Reproduisez pour les datasets suivants...
 - [Digits](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits) (en utilisant les données complètes)
 - [Iris](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)

... les expérimentations suivantes 

- Mise au point de plusieurs types de classifieurs (Perceptron, régression logistique, Arbre de décision, SVM). Pour chacun de ces types de classifieurs vous devrez :
 - Définir les hyper-paramètres à faire varier.
 - Evaluer et selectionner par Grid-Search l'ensemble des configurations possibles, en utilisant la Validation Croisée à 3 plis pour l'évaluation de la performance en généralisation. Vous pourrez vous inspirer d'un code tel que [celui-ci](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py) pour boucler sur les datasets et/ou les classifieurs.
- Ecrire sous forme d'un tableau récapitulatif les performances respectives (les meilleures obtenues) par chacun des modèles sur chacun des jeux de données (sur le test set).
- Donner des conclusions sur les résultats obtenus quand à la performance, la stabilité, la robustesse des familles de classifieurs utilisées, et les paramètres optimaux de chaque type de modèle.


  
 


Grid Search sur le Perceptron.
On décide de tester l'optimalité sur les paramètres suivants : alpha



In [17]:
from sklearn import grid_search

# On crée la liste des classifiers, pour plus de clarté dans le code :
classifiers = [
    Perceptron(),
    KNeighborsClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    SVC()
    ]

nom_classifiers = [
    "Perceptron",
    "KNeighborsClassifier",
    "LogisticRegression",
    "DecisionTreeClassifier",
    "SVC"
    ]

#Idem avec les paramètres des classifiers ci-dessus. On peut ainsi facilement y avoir accès et les modifier :
parameters = [
    {'alpha':[0.001, 0.01, 0.1, 1], 'eta0':[0.1, 0.05, 0.01, 0.005], 'n_iter':[2, 5, 10, 10, 100],
              'max_iter':[10, 20, 50, 100]},
    {'n_neighbors':[1, 2, 5, 10], 'p':[1, 2, 5], 'weights':['uniform', 'distance'], 'leaf_size':[20, 30, 40, 50]},
    {'C':[500, 1000, 2000], 'random_state':[0, 1, 2, 5]},
    {'max_depth':[None, 1, 2, 3, 4], 'splitter':['best'], 'max_leaf_nodes':[None, 2, 5]},
    {'kernel':['linear', 'rbf'], 'C':[1, 2, 5], 'gamma':[0.05, 0.1, 0.5, 1], 'random_state':[0, 1, 2]}
    ]


resultats_grid = []
i = 0
while i < len(classifiers):
    clf = grid_search.GridSearchCV(classifiers[i], parameters[i], cv=None)   # cv = None => 3 folders par défaut
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_test_std)
    #resultats_grid.append(
     #   "Classifier : %r => Meilleurs paramètres: %r, meilleur score: %r, misclassified samples: %r, accuracy: %.2f, score: %r"
      #                    % (classifiers[i], clf.best_params_, clf.best_score_, (y_test != y_pred).sum(),
       #                       accuracy_score(y_test, y_pred), classifiers[i].score(X_test_std,y_test)))
    
    print ('Classifier : %r' % (nom_classifiers[i]))
    print ('Best score en train : %r' % (clf.best_score_))
    print ('Best params : %r' % (clf.best_params_))
    print('Misclassified samples: %d' % (y_test != y_pred).sum())
    print('Accuracy: %.2f' % (accuracy_score(y_test, y_pred)))
    print ('Score en test : %r\n\n' % (clf.score(X_test_std,y_test)))
    i += 1
    
#print resultats_grid


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.

Classifier : 'Perceptron'
Best score en train : 0.8952380952380953
Best params : {'alpha': 0.001, 'max_iter': 50, 'eta0': 0.1, 'n_iter': 2}
Misclassified samples: 7
Accuracy: 0.84
Score en test : 0.84444444444444444


Classifier : 'KNeighborsClassifier'
Best score en train : 0.9523809523809523
Best params : {'n_neighbors': 5, 'weights': 'uniform', 'leaf_size': 20, 'p': 2}
Misclassified samples: 1
Accuracy: 0.98
Score en test : 0.97777777777777775


Classifier : 'LogisticRegression'
Best score en train : 0.9523809523809523
Best params : {'C': 500, 'random_state': 0}
Misclassified samples: 1
Accuracy: 0.98
Score en test : 0.97777777777777775


Classifier : 'DecisionTreeClassifier'
Best score en train : 0.9619047619047619
Best params : {'max_leaf_nodes': 5, 'max_depth': None, 'splitter': 'best'}
Misclassified samples: 1
Accuracy: 0.98
Score en test : 0.97777777777777775


Classifier : 'SVC'
Best score en train : 0.9523809523809523
Best params : {'kernel': 'rbf', 'C': 5, 'random_state': 0,

# Commentaires

Les résultats après gridsearch sont meilleurs qu'avec les premiers paramètres fixés. Elle fut donc utile !
Les résultats sont excellents (1 seule erreur en train en général, et un score en test proche de 98%).
On remarque que le perceptron est le classifier le moins efficace, ce qui n'est pas surprenant puisqu'il est beaucoup moins complexe que les autres classifiers.

Après avoir regardé les fiches sklearn des différents classifiers, on a fait varier les paramètres pertinents sur des valeurs du même ordre de grandeur que celle par défaut pour les grandeurs quantitatives. Pour les grandeurs qualitatives on a pu se permettre de les faire varier sur la plupart des valeurs possibles, l'ensemble étant fini.

In [19]:
digits = datasets.load_digits()

X = digits.data
y = digits.target

print digits.data.shape

print('Class labels:', np.unique(y))

(1797, 64)
('Class labels:', array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))


Splitting data into 70% training and 30% test data :

In [20]:
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

Standardizing the features :

In [21]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [22]:
from sklearn import grid_search

# On crée la liste des classifiers, pour plus de clarté dans le code :
classifiers = [
    Perceptron(),
    KNeighborsClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    SVC()
    ]

nom_classifiers = [
    "Perceptron",
    "KNeighborsClassifier",
    "LogisticRegression",
    "DecisionTreeClassifier",
    "SVC"
    ]

#Idem avec les paramètres des classifiers ci-dessus. On peut ainsi facilement y avoir accès et les modifier :
parameters = [
    {'alpha':[0.001, 0.01, 0.1, 1], 'eta0':[0.1, 0.05, 0.01, 0.005], 'n_iter':[2, 5, 10, 10, 100],
              'max_iter':[10, 20, 50, 100]},
    {'n_neighbors':[1, 2, 5, 10], 'p':[1, 2, 5], 'weights':['uniform', 'distance'], 'leaf_size':[20, 30, 40, 50]},
    {'C':[500, 1000, 2000], 'random_state':[0, 1, 2, 5]},
    {'max_depth':[None, 1, 2, 3, 4], 'splitter':['best'], 'max_leaf_nodes':[None, 2, 5]},
    {'kernel':['linear', 'rbf'], 'C':[1, 2, 5], 'gamma':[0.05, 0.1, 0.5, 1], 'random_state':[0, 1, 2]}
    ]


resultats_grid = []
i = 0
while i < len(classifiers):
    clf = grid_search.GridSearchCV(classifiers[i], parameters[i], cv=None)
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_test_std)
    #resultats_grid.append(
     #   "Classifier : %r => Meilleurs paramètres: %r, meilleur score: %r, misclassified samples: %r, accuracy: %.2f, score: %r"
      #                    % (classifiers[i], clf.best_params_, clf.best_score_, (y_test != y_pred).sum(),
       #                       accuracy_score(y_test, y_pred), classifiers[i].score(X_test_std,y_test)))
    
    print ('Classifier : %r' % (nom_classifiers[i]))
    print ('Best score en train : %r' % (clf.best_score_))
    print ('Best params : %r' % (clf.best_params_))
    print('Misclassified samples: %d' % (y_test != y_pred).sum())
    print('Accuracy: %.2f' % (accuracy_score(y_test, y_pred)))
    print ('Score en test : %r\n\n' % (clf.score(X_test_std,y_test)))
    i += 1
    
#print resultats_grid

Classifier : 'Perceptron'
Best score en train : 0.939538583929992
Best params : {'alpha': 0.001, 'max_iter': 20, 'eta0': 0.1, 'n_iter': 2}
Misclassified samples: 40
Accuracy: 0.93
Score en test : 0.92592592592592593


Classifier : 'KNeighborsClassifier'
Best score en train : 0.9721559268098647
Best params : {'n_neighbors': 5, 'weights': 'distance', 'leaf_size': 20, 'p': 2}
Misclassified samples: 17
Accuracy: 0.97
Score en test : 0.96851851851851856


Classifier : 'LogisticRegression'
Best score en train : 0.954653937947494
Best params : {'C': 1000, 'random_state': 0}
Misclassified samples: 35
Accuracy: 0.94
Score en test : 0.93518518518518523


Classifier : 'DecisionTreeClassifier'
Best score en train : 0.8273667462211615
Best params : {'max_leaf_nodes': None, 'max_depth': None, 'splitter': 'best'}
Misclassified samples: 94
Accuracy: 0.83
Score en test : 0.82592592592592595


Classifier : 'SVC'
Best score en train : 0.9729514717581543
Best params : {'kernel': 'linear', 'C': 1, 'random_

# Commentaire

Les résultats après gridsearch sont meilleurs qu'avec les premiers paramètres fixés. Elle fut donc utile ! Les résultats sont excellents mais moins bons que sur Iris. C'est du au fait que le data frame iris est très simple, là où Digits est plus complexe.
Sur Digits, DecisionTreeClassifier est de loin le moins bon classifier.
Après avoir regardé les fiches sklearn des différents classifiers, on a fait varier les paramètres pertinents sur des valeurs du même ordre de grandeur que celle par défaut pour les grandeurs quantitatives. Pour les grandeurs qualitatives on a pu se permettre de les faire varier sur la plupart des valeurs possibles, l'ensemble étant fini.